In [5]:
import scipy
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import pickle
import math

In [6]:
fDF = pd.read_csv('Data/featureTypes.txt', names=['featureID'])

In [7]:
print fDF.shape
print fDF['featureID'][0]
n = 10000
d = 3000

(3000, 1)
flavors raspberries cherries


In [8]:
trainDF = pd.read_csv('Data/trainData.txt', names = ['instanceID', 'featureID', 'value'], sep=' ')
YDF = pd.read_csv('Data/trainLabels.txt', names = ['label'])
valXDF = pd.read_csv('Data/valData.txt', names = ['instanceID', 'featureID', 'value'], sep=' ')
valYDF = pd.read_csv('Data/valLabels.txt', names = ['label'])
print trainDF.shape[0]

247847


In [9]:
W = np.random.rand(d)  #random.uniform(low=0.0, high=1.0,size = (d,))
B = 0 #np.zeros(n)
TempBArray = np.ones(n)
print W.shape
#print TempBArray
print np.count_nonzero(W)
print len(W)
print W

(3000,)
3000
3000
[ 0.38427511  0.18589934  0.59789612 ...,  0.83150823  0.28715457
  0.78962321]


In [10]:
trainDF[:5]

,instanceID,featureID,value
0,1,13,0.209
1,1,83,0.209
2,1,228,0.209
3,1,242,0.209
4,1,371,0.209


In [11]:
#tDF = csr_matrix(trainDF) 
#print tDF[:5]

In [12]:
#sdf = pd.SparseDataFrame(tDF)
#print sdf[:5]

In [13]:
# Will lead to negative index if re-running
trainDF['instanceID'] -= 1
trainDF['featureID'] -= 1
sMat = csr_matrix((trainDF['value'], (trainDF['featureID'], trainDF['instanceID'])))
valXDF['instanceID'] -= 1
valXDF['featureID'] -= 1
valX = csr_matrix((valXDF['value'], (valXDF['featureID'], valXDF['instanceID'])))
Y = YDF['label'].as_matrix().transpose()
#print Y.shape
valY = valYDF['label'].as_matrix().transpose()

In [14]:
#print sMat.shape
#print sMat.todense()
print sMat.max(), sMat.min()

0.728 0.0


In [15]:
#print sMat[:2]

In [16]:
X = sMat.copy()
print X.shape

(3000, 10000)


In [17]:
print len(W.nonzero())
print len(W)

1
3000


In [18]:
def initLamda(X, Y):
    YNorm = Y - float(Y.sum())/((float)(Y.shape[0]))
    return 2 * (abs(X * YNorm).max())
    
print initLamda(X, Y)

1271.869651


In [19]:
t = X.copy()
t.data **= 2
At = 2*t.sum(axis = 1)
print At

[[ 0.731852]
 [ 2.660682]
 [ 1.581472]
 ..., 
 [ 1.564942]
 [ 2.31796 ]
 [ 1.5809  ]]


In [20]:
redFact = 2

In [48]:
# change this to convergence condition

def rmse(input1, input2):
    out = input1 - input2
    #print out
    out **= 2
    out /= len(out)
    error = out.sum()
    return math.sqrt(error)


class Lasso:
    def __init__(self, X, Y, W, B, Lamda):
        self.X = X.copy()
        self.Y = Y.copy()
        self.W = W #.copy()  # Remove this copy later
        self.B = B #.copy()
        t = X.copy()
        t.data **= 2
        self.A = 2*t.sum(axis = 1)
        self.Lamda = Lamda
        self.delta = 0.001
        # Stores Lamda and respective RMSE
        self.trainrmse = []
        self.trainlamda = []
        self.valrmse = []
        self.vallamda = []
        self.NonZero = []
        
    def loss(self):
        return ((self.X.transpose() * self.W + self.B - self.Y) ** 2).sum() + self.Lamda * (abs(self.W)).sum()
        
    def fit(self):
        # Lamda = initLamda(self.X, self.Y)
        
        #print X.shape, W.shape
        #for epoch in range(100):
        oldLoss = self.loss()+2
        newLoss = self.loss()
        print 'Lamda: ', self.Lamda
        while oldLoss - newLoss > self.delta:
        #for i in range(1000):
            #print sMat.transpose() * W
            # 4.1.1
            #print t1[:5], t1.shape
            #print t1.shape, B.shape, Y.shape
            XTW = (self.X.transpose() * self.W)
            R = self.Y - (self.X.transpose() * self.W) - self.B
            
            # 4.1.2
            BOld = self.B
            self.B = np.full(n, (R + self.B).sum() / n) 
            #print self.B
            #print self.B.shape
            #self.B = (self.Y - XTW).sum() / n
            #print B.shape
            # 4.1.3
            R =  R + BOld - self.B
            #print 'RSHAPE', R.shape
            #print 'R', R
            #R = self.Y - (XTW + self.B)
            #print R.shape
            #print R[:5]
            # R = R.reshape(-1)
            for ik in range(0, d):
                # 4.1.4
                #ik = 0
                t = (self.X[ik].transpose() * self.W[ik]).toarray().reshape(-1)
                #print t
                #print t.shape
                #print R.shape
                Ck = 2*( self.X[ik] * (R + t)).sum()
                #print 'CK:', Ck
                # Update Weight
                WkOld = self.W[ik]
                #print 'OW: ', WkOld
                if Ck < -self.Lamda:
                    self.W[ik] = (Ck + self.Lamda) / self.A[ik]
                elif Ck > self.Lamda:
                    self.W[ik] = (Ck - self.Lamda) / self.A[ik]
                else:
                    self.W[ik] = 0
                #print 'W: ', WkOld, self.W[ik]
                #print W[ik]
                # 4.1.5
                # print self.W[ik], WkOld
                #print X[ik].toarray().reshape(-1).shape, R.shape
                R = R + self.X[ik].toarray().reshape(-1) * (WkOld - self.W[ik])
                #R = self.Y - (self.X.transpose() * self.W) + self.B
            oldLoss = newLoss
            newLoss = self.loss()
            #print oldLoss, newLoss, oldLoss - newLoss
            print 'LOSS:' , newLoss
            # End of feature vector iterator
    
    def saveModel(self, filename):
        pickle.dump(self, open( filename, "wb" ))
    
    def predict(self, X):
        return (X.transpose() * self.W + np.full(X.transpose().shape[0], self.B))
    
    def chooseCorrectLamda(self, delta = -1):
        oldLamda = self.Lamda
        if delta != -1:
            self.delta = delta
        self.fit()
        
        newRMSE = rmse(self.predict(self.X), self.Y)
        #self.TrainInfo.append([self.Lamda, newRMSE])
        self.trainrmse.append(newRMSE)
        self.trainlamda.append(self.Lamda)
        valRMSE = rmse(self.predict(valX), valY)
        self.valrmse.append(valRMSE)
        self.vallamda.append(self.Lamda)
        oldRMSE = valRMSE
        #print W
        #self.ValInfo.append([self.Lamda, valRMSE])
        self.NonZero.append((self.W != 0.0).sum())
        print 'Lamda: ', self.Lamda, 'RMSE: ', newRMSE, 'Val RMSE:' , valRMSE
        
        while self.Lamda >= 3: #oldRMSE >= valRMSE:
            oldLamda = self.Lamda
            self.Lamda /= redFact
            self.fit()
            oldRMSE = valRMSE
            #self.TrainInfo.append([self.Lamda, newRMSE])
            newRMSE = rmse(self.predict(self.X), self.Y)
            self.trainrmse.append(newRMSE)
            self.trainlamda.append(self.Lamda)
            valRMSE = rmse(self.predict(valX), valY)
            #self.ValInfo.append([self.Lamda, valRMSE])
            self.valrmse.append(valRMSE)
            self.vallamda.append(self.Lamda)
            self.NonZero.append(np.count_nonzero(self.W))
            #self.NonZero.append(self.W.toarray().count_nonzero())
            print 'Lamda: ', self.Lamda, 'RMSE: ', newRMSE, 'Val RMSE:' , valRMSE
            self.saveModel('optimal_saved_Model')
            
def loadModel(filename):
    return pickle.load(open(filename, "rb" ))

In [50]:
newModel = loadModel('./finalData/savedModel')
newModel.Lamda = 2.48
L = newModel.W

In [32]:
# Extract top 10 features
m = L.argsort()
print m

[ 740 2585  439 ...,  992 2664  973]


In [46]:
top10 = m[-10:]
print top10
less10 = m[:10]
#for i in less10:
#    print L[i]

[2468  933 1412 1835  811  358 2544  992 2664  973]
-11.1605322336
-6.94091162035
-6.57564610837
-6.10839472434
-5.67840297861
-5.62164283319
-5.56562977712
-5.43287124647
-5.35869477388
-5.06839904746


In [51]:
newModel.fit()
print newModel.W
L2 = newModel.W

Lamda:  2.48
LOSS: 33349.4423971
LOSS: 33244.8943273
LOSS: 33218.2413185
LOSS: 33208.6869985
LOSS: 33203.9299137
LOSS: 33201.0801947
LOSS: 33198.9182534
LOSS: 33196.9764207
LOSS: 33195.0892598
LOSS: 33193.1888694
LOSS: 33191.2524476
LOSS: 33189.2828108
LOSS: 33187.2991982
LOSS: 33185.3029935
LOSS: 33183.3084599
LOSS: 33181.3286594
LOSS: 33179.3772571
LOSS: 33177.4680177
LOSS: 33175.6079136
LOSS: 33173.8029762
LOSS: 33172.0569959
LOSS: 33170.3736
LOSS: 33168.7542004
LOSS: 33167.1993465
LOSS: 33165.7090577
LOSS: 33164.2826779
LOSS: 33162.9192388
LOSS: 33161.6172107
LOSS: 33160.3747783
LOSS: 33159.1900333
LOSS: 33158.060913
LOSS: 33156.9851921
LOSS: 33155.9609984
LOSS: 33154.9860242
LOSS: 33154.0579008
LOSS: 33153.174231
LOSS: 33152.3327571
LOSS: 33151.531413
LOSS: 33150.7682635
LOSS: 33150.0414978
LOSS: 33149.3493542
LOSS: 33148.6901369
LOSS: 33148.0623431
LOSS: 33147.4644111
LOSS: 33146.8948675
LOSS: 33146.3523399
LOSS: 33145.83557
LOSS: 33145.34328
LOSS: 33144.8743032
LOSS: 33144.42751

In [52]:
L2 = newModel.W
# Extract top 10 features
m2 = L2.argsort()
print m2


[ 740 2627 2585 ..., 2544  358  973]


In [53]:
top10 = m[-10:]
print top10
less10 = m[:10]

[2468  933 1412 1835  811  358 2544  992 2664  973]


In [66]:
#print fDF

In [80]:
t10 = top10[::-1]
print t10
l10 = less10[::-1]
print l10


[ 973 2664  992 2544  358  811 1835 1412  933 2468]
[ 536   94 1544 2042  446 1721  392  439 2585  740]


In [87]:
fDF['featureID'][l10].tolist()


['holds',
 'rest',
 'banana',
 'sparkler',
 'supported',
 'flavors enriched',
 'semillon',
 'liqueur',
 'cherry berry',
 'earns']

In [81]:
# 4.4.2
print '---TOP 10 Features---'
index = 0
for i in fDF['featureID'][t10]:
    print index+1, i
    index += 1
    pr

---TOP 10 Features---
1 spearmint
2 huge
3 lifesaver
4 big
5 stars
6 ageability
7 lemony
8 truly
9 acidity provides
10 apricots


In [82]:
print '---Least 10 Features---'
index = 0
for i in fDF['featureID'][l10]:
    print index+1, i
    index += 1

---Least 10 Features---
1 holds
2 rest
3 banana
4 sparkler
5 supported
6 flavors enriched
7 semillon
8 liqueur
9 cherry berry
10 earns
